## Preprocessing

In [130]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [131]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN","NAME"], axis=1)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [132]:
# Determine the number of unique values in each column.
unique_counts = application_df.nunique()
unique_counts

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [133]:
# Identify columns with more than 10 unique values
columns_with_more_than_10_unique_values = [col for col in application_df.columns if application_df[col].nunique() > 10]

# Display the number of data points for each unique value in those columns
for col in columns_with_more_than_10_unique_values:
    print(f"Column: {col}")
    print(application_df[col].value_counts())
    print()

Column: APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

Column: CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

Column: ASK_AMT
5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64



In [134]:
# Look at APPLICATION_TYPE value counts for binning
application_type_count = application_df["APPLICATION_TYPE"].value_counts()
application_type_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [135]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cut_off = 10
application_types_to_replace = list(application_type_count[application_type_count<cut_off].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: APPLICATION_TYPE, dtype: int64

In [136]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df["CLASSIFICATION"].value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [137]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_count_gt1 = classification_count[classification_count>1]
classification_count_gt1

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [138]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace
cut_off_classification = 10
classifications_to_replace = list(classification_count[classification_count<cut_off_classification].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Other       98
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
Name: CLASSIFICATION, dtype: int64

In [139]:
# Convert categorical data to numeric with `pd.get_dummies`
# Convert categorical data to numeric with pd.get_dummies
application_df_encoded = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'])

#application_df_encoded = pd.get_dummies(application_df, columns=["APPLICATION_TYPE", "CLASSIFICATION"])
application_df_encoded.head(10)


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T3,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,1,0
5,1,5000,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
6,1,31452,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,1,0
7,1,7508025,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
8,1,94389,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
9,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [140]:
# Split our preprocessed data into our features and target arrays

X = application_df_encoded.drop('IS_SUCCESSFUL', axis=1)
y = application_df_encoded['IS_SUCCESSFUL']


# Display the shapes of X and y
print("Shape of Features (X):", X.shape)
print("Shape of Target (y):", y.shape)
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)


Shape of Features (X): (34299, 71)
Shape of Target (y): (34299,)


In [141]:
y_test

10576    0
6439     0
11591    1
21566    0
345      0
        ..
10823    0
9039     1
13882    1
14492    1
24212    1
Name: IS_SUCCESSFUL, Length: 6860, dtype: int64

In [142]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [143]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_fetures = len(X_train_scaled[0])
hidden_nodes_layers1 = 5
hidden_nodes_layers2 = 9
hidden_nodes_layers3 = 5
hidden_nodes_layers4 = 5
hidden_nodes_layers5 = 9
hidden_nodes_layers6 = 5


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layers1, input_dim = number_input_fetures, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layers2, activation="relu"))

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layers3, activation="relu"))

#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layers4, activation="relu"))

#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layers5, activation="relu"))

#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layers6, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_35 (Dense)            (None, 5)                 360       
                                                                 
 dense_36 (Dense)            (None, 9)                 54        
                                                                 
 dense_37 (Dense)            (None, 5)                 50        
                                                                 
 dense_38 (Dense)            (None, 1)                 6         
                                                                 
Total params: 470 (1.84 KB)
Trainable params: 470 (1.84 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [144]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the callback to save the model's weights every five epochs
checkpoint_filepath = 'model_weights.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    save_freq='epoch',  # Save every epoch
    period=5  # Save every five epochs
)
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])

In [145]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
858/858 [==============================] - 1s 1ms/step - loss: 0.6367 - accuracy: 0.6600
Epoch 2/100
858/858 [==============================] - 1s 970us/step - loss: 0.5913 - accuracy: 0.7171
Epoch 3/100
858/858 [==============================] - 1s 1ms/step - loss: 0.5718 - accuracy: 0.7268
Epoch 4/100
858/858 [==============================] - 1s 1ms/step - loss: 0.5625 - accuracy: 0.7298
Epoch 5/100
858/858 [==============================] - 1s 992us/step - loss: 0.5582 - accuracy: 0.7293
Epoch 6/100
858/858 [==============================] - 1s 1ms/step - loss: 0.5560 - accuracy: 0.7301
Epoch 7/100
858/858 [==============================] - 1s 942us/step - loss: 0.5548 - accuracy: 0.7303
Epoch 8/100
858/858 [==============================] - 1s 923us/step - loss: 0.5540 - accuracy: 0.7305
Epoch 9/100
858/858 [==============================] - 1s 956us/step - loss: 0.5538 - accuracy: 0.7297
Epoch 10/100
858/858 [==============================] - 1s 897us/step - loss: 0.5

In [146]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5468 - accuracy: 0.7376 - 224ms/epoch - 1ms/step
Loss: 0.5467754006385803, Accuracy: 0.737609326839447


In [147]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

c:\Users\jherz\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
